In [5]:
import numpy as np
import pandas as pd
import overpy
import geopandas as gpd
import ee
import requests
from shapely.geometry import Polygon, box
import os


In [6]:
api = overpy.Overpass()

# Fetch all wastewater treatment plants within California's boundary
query = f"""
    area[admin_level=4]["name"="California"]->.searchArea;
    (
      way["man_made"="wastewater_plant"](area.searchArea);
    );
    (._;>;);
    out body;
    """

result = api.query(query)

# Store results in desired format
plants = {}

for way in result.ways:
    # Use the name of the plant or its ID if the name is not available
    plant_name = way.tags.get("name", f"Plant_{way.id}")

    # Extract nodes lat and lon without id
    nodes_coords = [(node.lon, node.lat) for node in way.nodes]

    # get rid of "Decimal"  in the coordinates
    nodes_coords = [tuple(map(float, i)) for i in nodes_coords]
    
    plants[plant_name] = nodes_coords


In [7]:
# print the first 5 plants
for key, value in list(plants.items())[:5]:
    print(key, value)

Plant_24298754 [(-121.8093956, 37.6930507), (-121.805817, 37.6930115), (-121.8056706, 37.6930099), (-121.8056706, 37.6904833), (-121.8058937, 37.6899807), (-121.8059778, 37.6894001), (-121.8060333, 37.6892735), (-121.8061508, 37.6892261), (-121.8071187, 37.6892498), (-121.8095244, 37.6896004), (-121.8096322, 37.6904367), (-121.809724, 37.6914042), (-121.8097223, 37.6915312), (-121.809522, 37.6915461), (-121.8093956, 37.6930507)]
Plant_24396910 [(-121.7836023, 36.8042341), (-121.7836343, 36.8037659), (-121.7833675, 36.8022232), (-121.7828697, 36.7990947), (-121.782329, 36.7983538), (-121.7828526, 36.7974329), (-121.7828135, 36.7971656), (-121.7827041, 36.7968984), (-121.7821745, 36.7964432), (-121.7809986, 36.7959414), (-121.7799, 36.7950548), (-121.7790588, 36.795261), (-121.7784065, 36.796072), (-121.7771877, 36.7992885), (-121.7776512, 36.8002576), (-121.7762693, 36.800938), (-121.7764839, 36.8015634), (-121.7779087, 36.8024362), (-121.7778655, 36.8026007), (-121.7777709, 36.8029603)

In [8]:
plants_with_official_name = {key: value for key, value in plants.items() if not key.startswith("Plant_")}
print(f"Number of official plants: {len(plants_with_official_name)}")

Number of official plants: 214


In [9]:
geoms = [Polygon(plants[key]) for key in plants]
df = gpd.GeoDataFrame({'WWTP_name': list(plants.keys()), 'geometry': geoms}, crs="EPSG:4326")
df

,WWTP_name,geometry
0,Plant_24298754,"POLYGON ((-121.80940 37.69305, -121.80582 37.6..."
1,Plant_24396910,"POLYGON ((-121.78360 36.80423, -121.78363 36.8..."
2,EBMUD Wastewater Treatment Plant,"POLYGON ((-122.29310 37.82285, -122.29238 37.8..."
3,Hyperion Wastewater Treatment Plant,"POLYGON ((-118.42914 33.91957, -118.43162 33.9..."
4,San José–Santa Clara Regional Wastewater Facility,"POLYGON ((-121.95539 37.43062, -121.95452 37.4..."
...,...,...
3099,Plant_1180546479,"POLYGON ((-122.36238 38.39425, -122.36120 38.3..."
3100,Plant_1186073992,"POLYGON ((-117.27039 34.25523, -117.27203 34.2..."
3101,Plant_1188491467,"POLYGON ((-121.52373 38.56623, -121.52238 38.5..."
3102,Plant_1201733978,"POLYGON ((-118.94698 37.64148, -118.94744 37.6..."


In [10]:
geoms_official_name = [Polygon(plants_with_official_name[key]) for key in plants_with_official_name]
df_official_name = gpd.GeoDataFrame({'WWTP_name': list(plants_with_official_name.keys()), 'geometry': geoms_official_name}, crs="EPSG:4326")
df_official_name

,WWTP_name,geometry
0,EBMUD Wastewater Treatment Plant,"POLYGON ((-122.29310 37.82285, -122.29238 37.8..."
1,Hyperion Wastewater Treatment Plant,"POLYGON ((-118.42914 33.91957, -118.43162 33.9..."
2,San José–Santa Clara Regional Wastewater Facility,"POLYGON ((-121.95539 37.43062, -121.95452 37.4..."
3,Sunnyvale Water Pollution Control Plant,"POLYGON ((-122.01815 37.41911, -122.01795 37.4..."
4,Burlingame Wastewater Treatment Plant,"POLYGON ((-122.35803 37.59173, -122.35745 37.5..."
...,...,...
209,County Club Wastewater Treatment Plant,"POLYGON ((-120.63039 35.21489, -120.63032 35.2..."
210,Riverside Water Quality Control Plant,"POLYGON ((-117.46432 33.96161, -117.46434 33.9..."
211,West County Wastewater - Water Quality & Resou...,"POLYGON ((-122.38026 37.96765, -122.38023 37.9..."
212,Jackson Rancheria Casino Resort Waste Water Plant,"POLYGON ((-120.73121 38.38433, -120.73104 38.3..."


How to authenticate: https://developers.google.com/earth-engine/guides/python_install#authentication

In [11]:
ee.Authenticate()


Successfully saved authorization token.


In [12]:
ee.Initialize()

In [13]:
df['geometry'].bounds

,minx,miny,maxx,maxy
0,-121.809724,37.689226,-121.805671,37.693051
1,-121.783634,36.795055,-121.776269,36.804234
2,-122.299808,37.822834,-122.291656,37.827277
3,-118.434659,33.919567,-118.425820,33.930754
4,-121.955388,37.428288,-121.940809,37.435635
...,...,...,...,...
3099,-122.362376,38.392237,-122.358658,38.394806
3100,-117.272211,34.253109,-117.269860,34.255232
3101,-121.523730,38.561855,-121.521200,38.566228
3102,-118.947438,37.637012,-118.942315,37.642072


NAIP dataset in Googel Earth Engine: https://developers.google.com/earth-engine/datasets/catalog/USDA_NAIP_DOQQ

ee.Geometry.Rectangle: https://github.com/gee-community/example-scripts/blob/master/Geometries/ee.Geometry.Rectangle.md

In [54]:
df_not_download = df.copy() # keep track of which plants have NOT been downloaded

In [39]:
# read in csv file that contains the names of the plants that have NOT been downloaded
# df_download = pd.read_csv("../30_result/trouble_download.csv")

In [55]:
# Create a directory to save images
download_directory = "../30_result/downloaded_images/"
if not os.path.exists(download_directory):
    os.mkdir(download_directory)
    
# Iterate over each wastewater treatment plant and fetch its image
for idx, row in df_not_download.iterrows():
    # 1. Get the bounding box for the current wastewater treatment plant
    bbox = row.geometry.bounds
    
    # 2. Determine the length and height of the WWTP and double them
    length = 2 * (bbox[2] - bbox[0])
    height = 2 * (bbox[3] - bbox[1])
    center_x = (bbox[0] + bbox[2]) / 2
    center_y = (bbox[1] + bbox[3]) / 2
    
    # Define the region of interest using the center and transformed dimensions
    roi = ee.Geometry.Rectangle([center_x - length / 2.5, \
                                center_y - height / 2.5, \
                                center_x + length / 2.5, \
                                center_y + height / 2.5])
    collection = ee.ImageCollection('USDA/NAIP/DOQQ') \
        .filterBounds(roi) \
        .filterDate(ee.Date('2020-01-01'), ee.Date('2023-01-01')) 
    
    # Get the first image from the collection
    image = collection.first().clip(roi)
    
    # Generate a thumbnail URL for the visualization
    try:
        url = image.getThumbURL({
            'bands': ['R', 'G', 'B'],
            'region': roi,
            'format': 'png',  # or 'jpeg'
            'max': 255,
            'min': 0
        })
    
        print(f"Download URL for {row['WWTP_name']}: {url}")
    
    except Exception as e:
        print(f"Failed to get image for {row['WWTP_name']}. Error: {e}")
        continue

    # Download the image
    try:
        response = requests.get(url)
    except Exception as e:
        # if response.status_code != 200:
        print(f"Failed to download image for {row['WWTP_name']}. Received status code: {response.status_code}")
        continue

    try:
        if 'image' in response.headers['Content-Type']:
            with open(os.path.join(download_directory, f"{row['WWTP_name']}.jpg"), 'wb') as file:  # Changing extension to jpg
                file.write(response.content)
            print(f"Downloaded image for {row['WWTP_name']}.")
            # delete the row in the dataframe
            df_not_download.drop(idx, inplace=True)
        else:
            print(f"Failed to download image for {row['WWTP_name']}. Received content: {response.text}")
    except Exception as e:
        print(f"Failed to download image for {row['WWTP_name']}. Error: {e}")
        continue

    # Export to Google Drive
    # task = ee.batch.Export.image.toDrive(
    #     image=image,
    #     description=row['WWTP_name'],
    #     folder="EE_images",
    #     fileNamePrefix=row['WWTP_name'] + "_image",
    #     region=roi.getInfo()["coordinates"],
    #     scale=1,  # You can adjust the scale (resolution) as needed
    #     fileFormat="GeoTIFF",
    #     maxPixels=1e13
    # )
    # task.start()

    # if idx == 19:
    #     break

Download URL for Plant_24298754: https://earthengine.googleapis.com/v1/projects/earthengine-legacy/thumbnails/5c9850caecfa35464250f62ebe863980-397f059faa525ccab7be6ea0d309164c:getPixels
Downloaded image for Plant_24298754.
Download URL for Plant_24396910: https://earthengine.googleapis.com/v1/projects/earthengine-legacy/thumbnails/597c2cf26b4e2a48f3a4de2de1296a19-07cd1dbdad83004d523af5314f22b558:getPixels
Downloaded image for Plant_24396910.
Download URL for EBMUD Wastewater Treatment Plant: https://earthengine.googleapis.com/v1/projects/earthengine-legacy/thumbnails/20e170ef8e7186b360115adfe6bcfe29-73209e117a2284ae81a26f726ac1ccbd:getPixels
Downloaded image for EBMUD Wastewater Treatment Plant.
Download URL for Hyperion Wastewater Treatment Plant: https://earthengine.googleapis.com/v1/projects/earthengine-legacy/thumbnails/304c0246b277e7cda36996b00ce10313-154846c311c0127b503a576126eee013:getPixels
Downloaded image for Hyperion Wastewater Treatment Plant.
Download URL for San José–Sant

In [37]:
df_not_download.shape

(1828, 2)

In [22]:
df.shape

(3104, 2)

In [38]:
df_not_download.to_csv("../30_result/trouble_download.csv", index=False)